In [2]:
# NAVEEN KUMAR RAJENDRAN-1002039449
#Machine Learning 6363 Assignment 3
# some basic imports
from scipy import io
import pandas as pd
import numpy as np
import math
import re
import matplotlib.pyplot as plt
import copy
from functools import partial
from multiprocessing.dummy import Pool as ThreadPool

%matplotlib inline
%load_ext autoreload
%autoreload 2

#loading training dataset
train = pd.read_csv('train.csv')
#loading test dataset
test = pd.read_csv('test.csv')

#displaying the shapes of the train and test data
print("train shape: {} test shape: {}".format(train.shape, test.shape))
#top 3 elements of train
train.head(3)

#top 3 elements of test
test.head(3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train shape: (1047, 11) test shape: (262, 11)


,Pclass,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,0,"Hart, Mr. Henry",male,NaN,0,0,394140,6.8583,NaN,Q
1,3,0,"Johnston, Master. William Arthur ""Willie""",male,NaN,1,2,W./C. 6607,23.4500,NaN,S
2,3,0,"Goodwin, Master. William Frederick",male,11.0,5,2,CA 2144,46.9000,NaN,S


In [3]:

Train = train.copy() # Using 'copy()' allows to clone the dataset, creating a different object with the same values


Titanic_data = [train, test]

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in Titanic_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in Titanic_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in Titanic_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column
for dataset in Titanic_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# Remove all NULLS in the Age column
for dataset in Titanic_data:
    Avg_age = dataset['Age'].mean()
    st_age = dataset['Age'].std()
    age_count = dataset['Age'].isnull().sum()
    null_age = np.random.randint(Avg_age - st_age, Avg_age + st_age, size=age_count)
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = null_age
    dataset['Age'] = dataset['Age'].astype(int)

# Define function to extract titles from passenger names
def get_title(name):
    title_srch = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_srch:
        return title_srch.group(1)
    return ""

for dataset in Titanic_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in Titanic_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in Titanic_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_map = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_map)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;

In [4]:
drop_ele = ['Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_ele, axis = 1)
test  = test.drop(drop_ele, axis = 1)

## Visualization of  the  titanic dataset ##

In [5]:
train.head(3)

,Pclass,Survived,Sex,Age,Parch,Fare,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,1,1,0,1,0,3,1,1,1,1,4
1,3,0,1,2,0,0,0,0,1,1,1
2,3,0,1,2,0,1,0,0,3,0,1


In [6]:
val = "Survived"
#var variable is selected to be dropped from the table
X = train.drop([val], axis=1)
X_test = test.drop([val], axis=1)
y = train[val]
y_test = test[val]
print("train: {}, test: {}".format(X.shape, X_test.shape))
print("train: {}, test: {}".format(y.shape, y_test.shape))
print((np.array(X)).shape[1])
print((np.array(y)).shape[0])

train: (1047, 10), test: (262, 10)
train: (1047,), test: (262,)
10
1047


In [7]:
def accuracy(val_1, val_2):
    #accuracy will be calculated as a sum when two values are equal and its divided my the length of the shape
    acc = np.sum(val_1 == val_2) / val_1.shape[0]
    return acc
print("Individaul Survived: {:.6f}, Individual Not Survivied: {:.6f}".format(y.sum()/len(y), 1-y.sum()/len(y)))

    

Individaul Survived: 0.387775, Individual Not Survivied: 0.612225


# Implementing Decision Tree 


In [8]:
# class for decision tree
class DecisionTree:
     #defining initialization for decision tree
    def __init__(self, criterion,max_depth,min_samples_split,min_samples_leaf,sampl_featr=False):
         #criterion for selecting  information_gain, gini_impurity
        if criterion == 'entropy':
            self.criterion = self._information_gain
        elif criterion == 'gini':
            self.criterion = self._gini_purification
        else:
            print('Criterion is ratio of gini or entropy')
        self._tree, self.sampl_featr,self.min_samples_split,self.max_depth, self.min_samples_leaf = None, sampl_featr, max_depth, min_samples_leaf, min_samples_split
        print(type(min_samples_leaf))
        
  # defining the fit method for the decision tree
    def fit(self, X, y, smpl_wt=None):
        if smpl_wt is None:
            smpl_wt = np.ones(X.shape[0]) / X.shape[0]
        else:
            smpl_wt = np.array(smpl_wt) / np.sum(smpl_wt)

        ftr_name, X, y = X.columns.tolist(), np.array(X), np.array(y)
        self._tree = self._build_tree(X, y, ftr_name, depth=1, sample_weights=smpl_wt)
        return self
        
   # defining entropy as static method 
    @staticmethod
    def entropy(y, smpl_wt=None):
        entrop, num = 0.0, y.shape[0]
        lab_count={}
        for i in range(num):
            if y[i] not in lab_count.keys():
                lab_count[y[i]]=0
            lab_count[y[i]]+=smpl_wt[i]
        for key in lab_count:
            entrop-=float(lab_count[key])/float(np.sum(smpl_wt))*np.log2(float(lab_count[key])/float(np.sum(smpl_wt)))
        return entrop

  #defining information gain 
    def _information_gain(self, X, y, indx, smpl_wt=None):
        info_gain, old_count, uni_val, new_count = 0, self.entropy(y, smpl_wt), np.unique(X[:,indx]), 0.0
        for value in uni_val:
            sub_X,sub_y, sub_smpl_wt=self._split_dataset(X, y, indx, value, smpl_wt)
            prob=np.sum(sub_smpl_wt)/float(np.sum(smpl_wt))
            new_count+=prob*self.entropy(sub_y, sub_smpl_wt)
        info_gain=old_count-new_count 
        return info_gain

   # defining gini_impurity as static method with labelcounts
    @staticmethod
    def gini_impurity(y, smpl_wt=None):
        gini_i, num = 1, y.shape[0]
        lab_count={}
        for i in range(num):
            if y[i] not in lab_count.keys():
                lab_count[y[i]]=0
            lab_count[y[i]]+=smpl_wt[i]
        for key in lab_count:
            gini_i -= (float(lab_count[key])/float(np.sum(smpl_wt))) ** 2
        return gini_i

  #defining gini_purification
    def _gini_purification(self, X, y, indx, smpl_wt=None):
        n_impurity, old_count, uni_val, new_count = 0, self.gini_impurity(y, smpl_wt), np.unique(X[:,indx]), 0.0
        for value in uni_val:
            sub_X,sub_y, sub_smpl_wt=self._split_dataset(X, y, indx, value, smpl_wt)
            new_count+=np.sum(sub_smpl_wt)/float(np.sum(smpl_wt))*self.gini_impurity(sub_y, sub_smpl_wt)
        n_impurity=old_count-new_count 
        return n_impurity

 #defining split dataset method
    def _split_dataset(self, X, y, indx, val, smpl_wt=None):
        rt, ft=[],X[:,indx]
        X=X[:,[i for i in range(X.shape[1]) if i!=indx ]]
        for i in range(len(ft)):
            if ft[i]==val:
                rt.append(i)
        sub_X, sub_y, sub_smpl_wt = X[rt,:], y[rt], smpl_wt[rt]
        return sub_X, sub_y, sub_smpl_wt

 #defining to choose best features
    def _choose_best_feature(self, X, y, smpl_wt=None):
        best_ftr_idx, n_ftr = 0, X.shape[1]
        if self.sampl_featr:
            max_ftr=max(1, min(n_ftr, int(np.round(np.sqrt(n_ftr)))))
            new_ftr=np.random.choice(n_ftr, max_ftr, replace=False)
            new_X=X[:, new_ftr]
        else:
            new_X=X
        n_new_ftr, best_gain_ct=new_X.shape[1], 0.0
        for i in range(n_new_ftr):
            info_gain_ct=self.criterion(new_X,y,i,smpl_wt)           
            if info_gain_ct > best_gain_ct:
                best_gain_ct, best_ftr_idx=info_gain_ct, i          
        return best_ftr_idx
        
 #defining majority vote as static method
    @staticmethod
    def majority_vote(y, smpl_wt=None):
        if smpl_wt is None:
            smpl_wt = np.ones(y.shape[0]) / y.shape[0]
        majority_lbl = y[0]
        dict_num={}
        for i in range(y.shape[0]):
            if int(y[i]) not in dict_num.keys():
                dict_num[y[i]]=smpl_wt[i]
            else:
                dict_num[y[i]] += smpl_wt[i]
        return max(dict_num, key=dict_num.get)

 #defining build tree for construction of decision tree
    # Update the _build_tree method
    def _build_tree(self, X, y, ftr_names, depth, sample_weights=None):
        mytree = dict()
        if len(ftr_names) == 0 or len(np.unique(y)) == 1 or depth >= self.max_depth or len(X) <= self.min_samples_leaf:
            return self.majority_vote(y, sample_weights)
    
        best_feature_idx = self._choose_best_feature(X, y, sample_weights)
        best_feature_name = ftr_names[best_feature_idx]
        ftr_names = ftr_names[:]
        ftr_names.remove(best_feature_name)
        mytree[best_feature_name] = {}
        uniq_vals = np.unique(X[:, best_feature_idx])
    
        for value in uniq_vals:
            sub_X, sub_y, sub_smpl_weights = self._split_dataset(X, y, best_feature_idx, value, sample_weights)
            mytree[best_feature_name][value] = self._build_tree(sub_X, sub_y, ftr_names, depth + 1, sub_smpl_weights)
    
        return mytree
 #defining predict method for decision trees
    def predict(self, X):
        # Defining classify method
        def _classify(tree, x):
            if not isinstance(tree, dict):
                return tree  

            ftr_name = next(iter(tree))  
            s_dict = tree[ftr_name]
            key = x[ftr_name]  # Access the feature value from the input data

            if key not in s_dict:
                key = np.random.choice(list(s_dict.keys()))

            valueOfKey = s_dict[key]

            # Recursively call _classify for nested trees
            return _classify(valueOfKey, x)

        # Predicting the output for each sample in X
        results = []
        for i in range(X.shape[0]):
            results.append(_classify(self._tree, X.iloc[i, :]))
        return np.array(results)

## Decision Tree Model Implementation  ##

In [9]:
 # implementing model of decision tree by calling decision tree class
dt = DecisionTree(criterion='gini', max_depth=4, min_samples_split=2,min_samples_leaf=4, sampl_featr=False)
dt.fit(X, y)
y_train_pred = dt.predict(X)
 # printing the accuracy of train of the model generated
print("Accuracy of the train set: {}".format(accuracy(y, y_train_pred)))
# printing the accuracy of test for the model generated
print("Accuracy of the test set: {}".format(accuracy(y_test, dt.predict(X_test))))



<class 'int'>
Accuracy of the train set: 0.8242597898758357
Accuracy of the test set: 0.8091603053435115


# Implementing Random Forest 

In [10]:
 #defining class random forest
class RandomForest:
    def __init__(self, tree_learner,num_trees,min_features=2020):
        np.random.seed(min_features)
        self.num_trees, self.weak_learner= num_trees, tree_learner
        self._estimators = [copy.deepcopy(self.weak_learner) for _ in range(self.num_trees)]

  #defining function to get bootstrap dataset
    def _get_bootstrap_dataset(self, X, y, i):
        X_arr, y_arr=np.array(X), np.array(y)
        idx=np.random.choice(X_arr.shape[0], X_arr.shape[0], replace=True)
        X_bootstrap= X.iloc[idx, :]
        y_bootstrap = y.iloc[idx]
        self._estimators[i].fit(X_bootstrap, y_bootstrap)

  #defining fit for random forest classifier
    def fit(self, X, y):
        self.labels, part_btstrap, pool=list(set(y)), partial(self._get_bootstrap_dataset, X, y), ThreadPool(self.num_trees)
        pool.map(part_btstrap, list(range(self.num_trees)))
        pool.close() and pool.join()
        return self

  # defining predict for random forest classifier 
    def predict(self, X):
        N = X.shape[0]
        y_pred = np.zeros(N)
        preds=np.array([estimator.predict(X) for estimator in self._estimators])
        pred_proba=np.array([(preds == label).mean(axis=0) for label in self.labels]).T
        y_pred=np.array(self.labels)[np.argmax(pred_proba, axis=1)]
        return y_pred


## Random forest Model Implementation  ##

In [11]:
# implementing model for random forest
base_learner = DecisionTree(criterion='gini', max_depth=2,min_samples_split=3, min_samples_leaf=4, sampl_featr=True)
r = RandomForest(tree_learner=base_learner, num_trees=50, min_features=2020)
r.fit(X, y)
#predicting the training set
y_train_pred = r.predict(X)
# printing the accuracy of train for the model generated
print("Accuracy of the train set: {}".format(accuracy(y, y_train_pred)))
# printing the accuracy of test for the model generated
print("Accuracy of the test set: {}".format(accuracy(y_test, r.predict(X_test))))



<class 'int'>
Accuracy of the train set: 0.8089780324737345
Accuracy of the test set: 0.8015267175572519


# Implementing AdaBoost(BOOSTING)

In [12]:
 # defining the boosting class
class Adaboost:
    # defining the initialization of method for boosting
    def __init__(self,weak_learner,num_learner,learning_rate,min_features=2020):
        np.random.seed(min_features)
        self.learning_rate= learning_rate
        self.weak_learner, self.num_learner = weak_learner, num_learner
        self._estimators = [copy.deepcopy(self.weak_learner) 
                            for _ in range(self.num_learner)]
        self._alphas = [1 
                        for _ in range(num_learner)]
    # Defining the fit method for boosting
    def fit(self, X, y):
        L = y.shape[0]
        w = np.ones(L) / L
        for i in range(self.num_learner):
            self._estimators[i].fit(X, y, smpl_wt=w)
            pred = self._estimators[i].predict(X)
            error = np.sum((pred != y) * w)
            alpha = 0.5 * np.log((1 - error) / max(error, 1e-10))
            self._alphas[i] = alpha
            w *= np.exp(-alpha * np.array(y) * pred) + 1e-4
            w /= 2 * np.sqrt(error * (1 - error))
        return self

    # Implementing the predict method for Adaboost
    def predict(self, X):
        N = X.shape[0]
        y_pred = np.zeros(N)
        for i in range(self.num_learner):
            y_pred += self._alphas[i] * self._estimators[i].predict(X)
        return np.sign(y_pred)



## Adaboost Model Implementation  ##

In [13]:
 # implementing the model for adabboost boosting classifier
base_learner = DecisionTree(criterion='gini', max_depth=4,min_samples_split=2, min_samples_leaf=3, sampl_featr=False)
ada = Adaboost(weak_learner= base_learner, num_learner=50,learning_rate=0.5, min_features=2020)
ada.fit(X, y)
#predicting the training set
y_train_pred = ada.predict(X)
# printing heaccuracy of train for the model generated
print("Accuracy of the train set: {}".format(accuracy(y, y_train_pred)))
# printing the accuracy of test for the model generated
print("Accuracy of the test set: {}".format(accuracy(y_test, ada.predict(X_test))))


<class 'int'>
Accuracy of the train set: 0.8118433619866284
Accuracy of the test set: 0.8129770992366412


# Table of results

In [20]:
print('The Model performance of Decision tree\n')
 # printing the accuracy of the model generated   
print("Accuracy of the train set: {}".format(accuracy(y, y_train_pred)))
 # printing the accuracy of the model generated   
print("Accuracy of the test set: {}".format(accuracy(y_test, dt.predict(X_test))))
print('\nThe Model performance of Random Forest\n')
 # printing the accuracy of the model generated   
print("Accuracy of the train set: {}".format(accuracy(y, y_train_pred)))
 # printing the accuracy of the model generated   
print("Accuracy of the test set: {}".format(accuracy(y_test, r.predict(X_test))))
print('\nThe Model performance of Boosting(ADAboost)\n')
 # printing the accuracy of the model generated   
print("Accuracy of the train set: {}".format(accuracy(y, y_train_pred)))
 # printing the accuracy of the model generated   
print("Accuracy of the test set: {}".format(accuracy(y_test, ada.predict(X_test))))


The Model performance of Decision tree

Accuracy of the train set: 0.8118433619866284
Accuracy of the test set: 0.8091603053435115

The Model performance of Random Forest

Accuracy of the train set: 0.8118433619866284
Accuracy of the test set: 0.8015267175572519

The Model performance of Boosting(ADAboost)

Accuracy of the train set: 0.8118433619866284
Accuracy of the test set: 0.8129770992366412


## On comparing the 3 models AdaBoost(Boosting) provides more Accuracy than the other 2 models decision tree and Random forest ##